## Optimization packages playground

In [8]:
# Define f
a = 1.0
b = 100.0

my_f_calls = [0]

"Rossenbrok's function"
function rosenbrock(x::Vector{<:Real}, a::Real, b::Real, my_f_calls=my_f_calls) 
    push!(my_f_calls, 1)
    (a - x[1])^2 + b * (x[2] - x[1]^2)^2
end
"Closure"
f = x -> rosenbrock(x, a, b, my_f_calls);

"Rossenbrok's function gradient"
function g!(G, x)
    G[1] = -2.0 * (1.0 - x[1]) - 400.0 * (x[2] - x[1]^2) * x[1]
    G[2] = 200.0 * (x[2] - x[1]^2)
end

"Rossenbrok's Hessian matrix"
function 𝐻rosenbrock!(H, x)
    H[1, 1] = 2.0 - 400.0 * x[2] + 1200.0 * x[1]^2
    H[1, 2] = -400.0 * x[1]
    H[2, 1] = -400.0 * x[1]
    H[2, 2] = 200.0
end;
    
"Analytic solution"
xₘᵢₙ = [a, a^2];


In [49]:
using Plots; pyplot()
x  = [-2:0.05:2;]
y = [-1:0.05:3;]
z = [f([x,y]) for x in x, y in y]

minZ = minimum(z[:]);  
maxZ = maximum(z[:]);

COL = append!([colorant"blue",colorant"lime"],range(colorant"yellow",colorant"red",length=20))
c =  minZ .+ (maxZ-minZ).*log.(1 .+z .- minZ) ./ log(1+maxZ-minZ)

Plots.plot(x,y,z,st=:surface,color=cgrad(COL,scale=:exp),#cgrad(:jet,c),
xlabel = "x",ylabel="y",zlabel="f(x,y)",zguidefontrotation=45,camera=(-30,30))

### Optim.jl: Gradient required

In [2]:
using Optim

# Box 
lower = [0.3, -2.1]
upper = [30.0, 40.0]

# Initial guess
x₀ = [0.4,0.5];

### Available algos

In [3]:
inner_optimizer = GradientDescent()
inner_optimizer = ConjugateGradient()
momentum  = 10
inner_optimizer = LBFGS(m = momentum)

LBFGS{Nothing, LineSearches.InitialStatic{Float64}, LineSearches.HagerZhang{Float64, Base.RefValue{Bool}}, Optim.var"#20#22"}(10, LineSearches.InitialStatic{Float64}
  alpha: Float64 1.0
  scaled: Bool false
, LineSearches.HagerZhang{Float64, Base.RefValue{Bool}}
  delta: Float64 0.1
  sigma: Float64 0.9
  alphamax: Float64 Inf
  rho: Float64 5.0
  epsilon: Float64 1.0e-6
  gamma: Float64 0.66
  linesearchmax: Int64 50
  psi3: Float64 0.1
  display: Int64 0
  mayterminate: Base.RefValue{Bool}
, nothing, Optim.var"#20#22"(), Flat(), true)

In [9]:
# Set tolerances
#Ncalls to f 
outer_iters = 40
#Ncalls to alg step
iters = 30
# gradient tolerance 
GTOL = 1e-12
options = Optim.Options(g_tol = 1e-12, outer_iterations = outer_iters, iterations = iters, store_trace = true)

# Forward Diff
res_fdif = optimize(f, lower, upper, x₀, Fminbox(inner_optimizer),options, autodiff = :finite )
res = optimize(f, g!, lower, upper, x₀, Fminbox(inner_optimizer))

function show_results(res)
    @show summary(res)
    @show minimum(res)
    @show Optim.minimizer(res)
    @show Optim.iterations(res)
    @show Optim.iteration_limit_reached(res)
    # @show length(Optim.f_trace(res))
    @show Optim.f_calls(res)
    @show Optim.converged(res)
end


println("-------ForwardDiff with ∇--------")
show_results(res_fdif)

println("-------Grdient--------")
show_results(res)

println("-------My fcalls--------")
@show sum(my_f_calls)


-------ForwardDiff with ∇--------


summary(res) = "Fminbox with L-BFGS"
minimum(res) = 5.361093300320716e-17


Optim.minimizer(res) = [0.9999999926780513, 0.9999999853561207]
Optim.iterations(res) = 4
Optim.iteration_limit_reached(res) = false
Optim.f_calls(res) = 78
Optim.converged(res) = true
-------Grdient--------
summary(res) = "Fminbox with L-BFGS"
minimum(res) = 1.383848755571263e-22
Optim.minimizer(res) = [1.0000000000117624, 1.0000000000235425]
Optim.iterations(res) = 4
Optim.iteration_limit_reached(res) = false
Optim.f_calls(res) = 74
Optim.converged(res) = true
-------My fcalls--------
sum(my_f_calls) = 466


466

### Optim.jl: Gradient free 

In [43]:
inner_optimizer = NelderMead()
res_nmead = optimize(f, lower, upper, x₀, Fminbox(inner_optimizer),options)


println("-------Nelder Mead--------")
show_results(res_nmead)

-------Nelder Mead--------
summary(res) = "Fminbox with Nelder-Mead"
minimum(res) = 3.0143331585332986e-9
Optim.minimizer(res) = [0.9999499855832705, 0.9999022383782905]
Optim.iterations(res) = 5
Optim.iteration_limit_reached(res) = true
Optim.f_calls(res) = 350
Optim.converged(res) = true


true

## Black Box Optim

In [12]:
using BlackBoxOptim


method = :probabilistic_descent
method = :de_rand_1_bin_radiuslimited
method = :adaptive_de_rand_1_bin_radiuslimited

my_f_calls = [0]

good_guess = [3.0, 7.2]
two_good_guesses = [[3.0, 7.2], [3.0, 7.2]]
res = bboptimize(f, two_good_guesses; 
SearchRange = [(lower[1], upper[1]), (lower[2], upper[2])], 
 Method = method,
 MaxTime = 10,
 MaxFuncEvals = 20,
 FitnessScheme  = MinimizingFitnessScheme, # fitness scheme to be used
 FitnessTolerance = 1e-8, # fitness scheme to be used
 MaxSteps = 10000,
 TraceMode = :verbose,
 PopulationSize = 60,
 TargetFitness = nothing, # optimal (target) fitness, if known
 SaveTrace      = true,
 SaveFitnessTraceToCsv = false,
 SaveParameters = true,
 MaxStepsWithoutProgress = 10000,
 RngSeed        = 1234,   # The specific random seed to set before any random numbers are generated. The seed is randomly selected if RandomizeRngSeed is true, and this parameter is updated with its actual value.
 RandomizeRngSeed = false,
 )

# Maximum time allowed in seconds. 

# Number of function evaluations allowed, this needs  MaxTime to be false then the MaxFuncEvals plays a role
println("-------BBO Otpim--------")
# Access to the solution 
@show best_candidate(res)
@show best_fitness(res)
@show minimum(res)
@show iteration_converged(res)
@show res.stop_reason
@show fitness_scheme(res)
@show res.iterations
@show res.f_calls

println("-------my_f_calls--------")

@show sum(my_f_calls)
# writetable(res)

Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64}, RadiusLimitedSelector, BlackBoxOptim.AdaptiveDiffEvoRandBin{3}, RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
DE modify state:

Optimization stopped after 19532 steps and 0.03 seconds
Termination reason: Too many steps (101) without any function evaluations (probably search has converged)
Steps per second = 658064.81
Function evals per second = 630100.76
Improvements/step = Inf
Total function evaluations = 18702


Best candidate found: [1.0, 1.0]

Fitness: 0.000000000

-------BBO Otpim--------
best_candidate(res) = [1.0, 1.0]
best_fitness(res) = 0.0
minimum(res) = [1.0, 1.0]
iteration_converged(res) = true
res.stop_reason = "Too many steps (101) without any function evaluations (probably search has converged)"
fitness_scheme(res) = ScalarFitnessScheme{true}()
res.iterations = 19532
res.f_calls = 18702
-------my_f_calls--------
sum(my_f_calls) = 18703


18703